# Рекомендация тарифов

**Дано:** оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

Есть таблица с признаками:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Задача:** построить модель для задачи классификации, которая выберет подходящий тариф, с максимально большим значением accuracy. 


**План исследования:**
 1. Обзор данных.
 2. Разделение данных на обучающую, валидационную и тестовую выборки.
 3. Исследование качества разных моделей со сменой гиперпараметров.
 4. Проверка качества модели на тестовой выборке.
 5. Проверка модели на вменяемость.
 6. Общий вывод.

## 1. Обзор данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
try:
    users = pd.read_csv('/Users/galina//Desktop/учёба/users_behavior.csv')
except:
    users = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
users.head(5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
#смотрю дубликаты:
users.duplicated().sum()

0

In [6]:
#меняю тип данных с float на int в столбцах `calls` и `messages`:
users['calls'] = users['calls'].astype('int')
users['messages'] = users['messages'].astype('int')
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


### Выводы по п.1. Обзор данных:

1. В таблице 3214 объекта, пропусков нет.
2. Поменяла тип данных с float на int в столбцах `calls` и `messages`, хотя это никак не повлияет на исследование. Прочая предобработка данных не понадобится, сделана ранее.
4. Целевой признак - `is_ultra` 

## 2. Разделение данных на обучающую, валидационную и тестовую выборки


In [7]:
#создаю переменные для признаков и целевого признака:
features = users.drop(['is_ultra'], axis=1)
target = users['is_ultra']

#разделяю данные на обучающую и валидационную выборки в соотношении 60/40:
features_train, features_valid, target_train, target_valid = train_test_split(features,
                                                                              target, test_size=0.4,
                                                                              random_state=12345)

#разделяю валидационную выборку на валидационную и тестовую в соотношении 20/20:
features_test, features_valid, target_test, target_valid = train_test_split(features_valid,
                                                                            target_valid, test_size=0.5,
                                                                            random_state=12345)

In [8]:
#смотрю размеры получившихся выборок:
for i in [features_train, features_valid, features_test]:
    print(i.shape)

for i in [target_train, target_valid, target_test]:
    print(i.shape)

(1928, 4)
(643, 4)
(643, 4)
(1928,)
(643,)
(643,)


Получились обучающая, валидационная и тестовая выборки на 60%, 20% и 20% от первоначального датасета соответственно.

## 3. Исследование качества разных моделей со сменой гиперпараметров

### 3.1. Дерево решений

In [9]:
#исследую аccuracy валидационной выборки:
best_acc_dt_vs = 0
best_depth_dt_vs = 0
for depth in range(1, 10):
    model_dt_vs = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_dt_vs.fit(features_train, target_train)
    predictions_valid_dt_vs = model_dt_vs.predict(features_valid)
    acc_dt_vs = accuracy_score(target_valid, predictions_valid_dt_vs)
    print('Accuracy на шаге', depth, '=', acc_dt_vs)
    if acc_dt_vs > best_acc_dt_vs:
        best_acc_dt_vs = acc_dt_vs
        best_depth_dt_vs = depth
        
print()
print('Accuracy валидационной выборки по алгоритму дерево решений =', best_acc_dt_vs)
print('Максимальная глубина =', best_depth_dt_vs)

Accuracy на шаге 1 = 0.7356143079315708
Accuracy на шаге 2 = 0.7744945567651633
Accuracy на шаге 3 = 0.7791601866251944
Accuracy на шаге 4 = 0.7744945567651633
Accuracy на шаге 5 = 0.7838258164852255
Accuracy на шаге 6 = 0.776049766718507
Accuracy на шаге 7 = 0.7993779160186625
Accuracy на шаге 8 = 0.7931570762052877
Accuracy на шаге 9 = 0.7807153965785381

Accuracy валидационной выборки по алгоритму дерево решений = 0.7993779160186625
Максимальная глубина = 7


### 3.2. Случайный лес

In [10]:
#исследую аccuracy валидационной выборки:
best_acc_dt_vs = 0
best_depth_dt_vs = 0
best_est_dt_vs = 0
for depth in range(1, 10):
    for est in range(1, 100):
        model_dt_vs = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_dt_vs.fit(features_train, target_train)
        predictions_valid_dt_vs = model_dt_vs.predict(features_valid)
        acc_dt_vs = accuracy_score(target_valid, predictions_valid_dt_vs)
        if acc_dt_vs > best_acc_dt_vs:
            best_acc_dt_vs = acc_dt_vs
            best_depth_dt_vs = depth
            best_est_dt_vs = est

print('Accuracy валидационной выборки по алгоритму случайный лес =', best_acc_dt_vs)
print('Максимальная глубина =', best_depth_dt_vs)
print('Число деревьев =', best_est_dt_vs)

Accuracy валидационной выборки по алгоритму случайный лес = 0.8149300155520995
Максимальная глубина = 9
Число деревьев = 14


### 3.3. Логистическая регрессия

In [11]:
#исследую аccuracy валидационной выборки:
model_lr_vs = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
model_lr_vs.fit(features_valid, target_valid)
predictions_lr_vs = model_lr_vs.predict(features_valid) 
acc_lr_vs = accuracy_score(target_valid, predictions_lr_vs) 
        
print('Accuracy валидационной выборки по алгоритму линейная регрессия =', acc_lr_vs)

Accuracy валидационной выборки по алгоритму линейная регрессия = 0.6858475894245724


### Выводы по п.3. Исследование качества разных моделей со сменой гиперпараметров:

Поскольку accuracy валидационной выборки по алгоритму:
- дерево решений = 0.799
- случайный лес = 0.815
- линейная регрессия = 0.686

наиболее качественные результаты прогнозирует случайный лес.

## 4. Проверка качества модели на тестовой выборке

In [13]:
model_dt_test = RandomForestClassifier(random_state=12345, n_estimators=100, max_depth=20)
model_dt_test.fit(features_train, target_train)
predictions_dt_test = model_dt_test.predict(features_test)
acc_dt_test = accuracy_score(target_test, predictions_dt_test)

print('Accuracy тестовой выборки по алгоритму случайный лес =', acc_dt_test)

Accuracy тестовой выборки по алгоритму случайный лес = 0.7884914463452566


### Выводы по п.4. Проверка качества модели на тестовой выборке:

Поскольку accuracy валидационной выборки по алгоритму:
- дерево решений = 0.799
- случайный лес = 0.813
- линейная регрессия = 0.686,

а accuracy тестовой выборки по алгоритму случайный лес = 0.788:

Можно утверждать, что случайный лес прогнозирует качественный результат.

## 5. Проверка модели на вменяемость

Использую DummyClassifier для проверки модели на вменяемость по 3 стратегиям:
- uniform, так как генерирует прогнозы равномерно в случайном порядке
- stratified, так как генерирует прогнозы с учетом распределения классов обучающей выборки
- most_frequent, так как всегда предсказывает наиболее частую метку в обучающем наборе

In [15]:
for strategy in ['uniform', 'most_frequent', 'stratified']:
    dummy = DummyClassifier(strategy=strategy)
    dummy.fit(features_train, target_train)
    dummy_valid = dummy.score(features_valid, target_valid) 
    dummy_test = dummy.score(features_test, target_test) 
    print('Accuracy случайной модели на валидационной выборке для стратегии', i,'=', dummy_valid)
    print('Accuracy случайной модели на тестовой выборке для стратегии', i,'=', dummy_test)
    print()

Accuracy случайной модели на валидационной выборке для стратегии uniform = 0.47433903576982894
Accuracy случайной модели на тестовой выборке для стратегии uniform = 0.48367029548989116

Accuracy случайной модели на валидационной выборке для стратегии most_frequent = 0.6842923794712286
Accuracy случайной модели на тестовой выборке для стратегии most_frequent = 0.7060653188180405

Accuracy случайной модели на валидационной выборке для стратегии stratified = 0.5458786936236392
Accuracy случайной модели на тестовой выборке для стратегии stratified = 0.5894245723172629



### Выводы по п.5. Проверка модели на вменяемость:

Accuracy случайной модели на тестовой выборке - примерно 0.6. Accuracy случайного леса на тестовой выборке - 0.788. Следовательно, рассмотренная модель вменяема. 

## 6. Общий вывод 

На основе данных по клиентам оператора мобильной связи «Мегалайн» можно построить систему, которая выберет подходящий тариф. Для этого надо использовать алгоритм случайный лес, так как он дает максимально большое значение accuracy.